# Prueba Python para el análisis de datos

### Daniel Rojas López


In [ ]:
import psycopg2 
import pandas as pd 
import openpyxl
from sqlalchemy import create_engine

def coneccionBD(): 
    # Conéctate a PostgreSQL (sin especificar una base de datos) 
    conn = psycopg2.connect(database="classicmodels", #Es el nombre de la base de datos a la que te quieres conectar, postgres es por defecto 
                                user="postgres", #Es el nombre de usuario que tiene permisos para conectarse a esa base de datos. 
                                password="postgres",  
                                host="localhost", #Indica dónde está alojada la base de datos. "localhost" significa que está en el mismo computador donde estás ejecutando el script de Python. 
                                port="5432")     
    print("Base de datos conectada exitosamente.") 
    #cursor = conn.cursor()    
    # Crear la base de datos si no existe
    #cursor.execute("DROP DATABASE IF EXISTS northwind2;")  # Elimina si ya existe
    #cursor.execute("CREATE DATABASE northwind2;")  # Crea la base de datos
    return conn 
    
def cargarBD(conn):

    #cargar el script sql para carga la base de datos
    cursor = conn.cursor()

    sql_file_path = "classicmodels.sql"

    # Leer el archivo SQL y ejecutarlo
    with open(sql_file_path, "r", encoding="utf-8") as sql_file:
        print("Leyendo el archivo SQL..."+sql_file_path)  
        sql_script = sql_file.read()  # Lee todo el contenido del archivo SQL
        cursor.execute(sql_script)  # Ejecuta el script
    
    conn.commit()  # Confirma los cambios
    cursor.close()
    conn.close()

    print("Base de datos importada exitosamente.")
    return conn
    
def obtener_tablas(conn):

    cursor = conn.cursor()
    # Ejecutar una consulta simple
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
    tables = cursor.fetchall()

    cursor.close()
    conn.close()
    return tables

def leer_tabla(conn, tabla):
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {tabla};")
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    cursor.close()
    return df

    
 
#1. Genera una función llamada leer_tabla(tabla, engine) y utilízala para leer tablas completas desde la base de datos en dataframes independientes. Utilizando esta función, importa las siguientes tablas:
#- order
#- orderdetails
#- customers
#- products
#- employees


conn = coneccionBD()
#cargarBD(conn)

#tablas=obtener_tablas(conn)

dforder=leer_tabla(conn, "orders")
dforder=dforder.drop_duplicates()
dforderdetails=leer_tabla(conn, "orderdetails")
dforderdetails=dforderdetails.drop_duplicates()
dfcustomers=leer_tabla(conn, "customers")
dfproducts=leer_tabla(conn, "products")
dfemployees=leer_tabla(conn, "employees")

#print(dforder)
#print(dforderdetails)
#print(dfcustomers)
#print(dfproducts)
#print(dfemployees)


# 2. Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el parámetro validate para asegurar la integridad referencial.

#hacer el cruce entre los dataframes
dfjoinOrderOrderDetails= dforder.merge(dforderdetails, on="orderNumber", validate="one_to_many")
dfResultado=dfjoinOrderOrderDetails.drop_duplicates()
#dfResultado.to_excel("resuktado.xlsx", index=True)

dfResultadoCustomers= dfResultado.merge(dfcustomers, on="customerNumber", validate="many_to_one")
dfResultado2=dfResultadoCustomers.drop_duplicates()
#dfResultado2.to_excel("resuktado2.xlsx", index=True)

dfResultado2Products= dfResultado2.merge(dfproducts, on="productCode", validate="many_to_one")
dfResultado3=dfResultado2Products.drop_duplicates()
#dfResultado3.to_excel("resuktado3.xlsx", index=True)

#no tengo indice con que vincular
#dfResultado3Employees= dfResultado3.merge(dfemployees, on="employeeNumber", validate="many_to_one")
#dfResultado4=dfResultado3Employees.drop_duplicates()

# 3. Agrega las siguientes columnas, considerando su nombre y la fórmula asociada
# - venta: quantityOrdered*priceEach
# - costo: quantityOrdered*buyPrice

# Agregamos las nuevas columnas
dfResultado3["venta"] = dfResultado3["quantityOrdered"] * dfResultado3["priceEach"]
dfResultado3["costo"] = dfResultado3["quantityOrdered"] * dfResultado3["buyPrice"]
dfResultado3["ganancia"] = dfResultado3["venta"] - dfResultado3["costo"]

#dfResultado3.to_excel("conNuevaColumnas.xlsx", index=True)




# 4. ¿Cuál fue el total de ventas por línea de productos? Incluye una fila de totales.

dfventasPorLinea=dfResultado3.groupby("productLine")["venta"].sum()
#+dfventasPorLinea

# 5. ¿Cuántos clientes distintos hicieron compras?

dfclientesDistintos=dfResultado3.groupby("customerNumber")["customerNumber"].count()
#+dfclientesDistintos


# 6. ¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?

# Verificamos qué customerNumbers de la tabla customers no están en orders
clientes_sin_compras = dfcustomers[~dfcustomers['customerNumber'].isin(dforder['customerNumber'])]

# Contamos los clientes sin compras
cantidad = len(clientes_sin_compras)
print(f"Hay {cantidad} clientes que no han realizado compras.")


# 7. Se solicita la creación de dos reportes, que respondan las preguntas dadas

# ● ¿Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada top_10_clientes_2005, en la que se especifique el nombre del cliente y su correspondiente venta, costo y ganancia.

# Filtrar órdenes del 2005
df_2005 = dforder[dforder['orderDate'].dt.year == 2005].copy()

# Unir con orderdetails para obtener los montos
df_ventas = pd.merge(
    df_2005[['orderNumber', 'customerNumber']],
    dforderdetails[['orderNumber', 'quantityOrdered', 'priceEach']],
    on='orderNumber'
)

# Calcular venta bruta por línea de orden
df_ventas['venta_bruta'] = df_ventas['quantityOrdered'] * df_ventas['priceEach']

# Agrupar por cliente y sumar ventas
ventas_por_cliente = df_ventas.groupby('customerNumber')['venta_bruta'].sum().reset_index()

# Unir con la tabla de clientes para obtener el nombre
ventas_por_cliente = pd.merge(
    ventas_por_cliente,
    dfcustomers[['customerNumber', 'customerName']],
    on='customerNumber'
)

# Ordenar y seleccionar top 10
top_10_clientes = ventas_por_cliente.sort_values('venta_bruta', ascending=False).head(10)

# Crear DataFrame final
df_top_10 = top_10_clientes[['customerName', 'venta_bruta']].rename(columns={
    'customerName': 'nombre_cliente',
    'venta_bruta': 'venta_total'
})

# Guardar el DataFrame en PostgreSQL
df_top_10.to_sql('top_10_clientes_2005', conn, if_exists='replace', index=False)

print("Los 10 clientes con mayores ventas en 2005 son:")
print(df_top_10)


# ● ¿Cuál fue el top 10 de artículos más vendidos durante el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada top_10_prod   uctos_2005, en la que se especifique el nombre del producto y su correspondiente venta, costo y ganancia.  Para este punto debes aplicar el principio DRY, por lo que se deben utilizar funciones parrealizar el filtrado por fechas, generar tablas pivote y escribir tabla en Postgre. Las funciones deben estar en un archivo separado llamado funciones.py y ser importadas al Jupyter Notebook. En este archivo se debe incluir:







# ● Una función que permita filtrar un DataFrame por fechas, indicando dataframe, columna para filtrar, fecha inicio y fecha fin. La función debe retornar un DataFrame.
# ● Una función que permita generar reportes dependiendo de parámetros de entrada como dataframe, filas, columnas, valores y medida (funcion_agrupadora). Utilizar fill_value = 0. Esta función debe retornar un DataFrame pivotado.
# ● Una función que permita escribir en la base de datos a través del guardado de un DataFrame dependiendo de parámetros de entrada como DataFrame, nombre de la tabla, engine y comportamiento en caso de que exista la tabla (if_exists).
    

Base de datos conectada exitosamente.
Hay 24 clientes que no han realizado compras.
